In [19]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

Elimino primero el ResponseId (index), la columna de Check (pregunta en la encuesta para verificar que el encuestado presta atencion a lo que responde) y ConvertedCompYearly (conversion a dolares americanos de los que gana, lo elimino porque tratare solo EUR)

In [20]:
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [21]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df24.pickle', 'rb') as archivo:
    df = pickle.load(archivo)

In [22]:
df = df.drop(columns=['Check', 'ConvertedCompYearly'])

In [23]:
# y = df['CompTotal']

In [24]:
# df = df.drop(columns=['CompTotal'], axis=1) 

In [25]:
df.head()

,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,TechDoc,YearsCode,YearsCodePro,DevType,OrgSize,PurchaseInfluence,BuyNewTool,BuildvsBuy,TechEndorse,Country,Currency,CompTotal,LanguageHaveWorkedWith,LanguageWantToWorkWith,LanguageAdmired,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,DatabaseAdmired,PlatformHaveWorkedWith,PlatformWantToWorkWith,PlatformAdmired,WebframeHaveWorkedWith,WebframeWantToWorkWith,WebframeAdmired,EmbeddedHaveWorkedWith,EmbeddedWantToWorkWith,EmbeddedAdmired,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,MiscTechAdmired,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,ToolsTechAdmired,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,NEWCollabToolsAdmired,OpSysPersonal use,OpSysProfessional use,OfficeStackAsyncHaveWorkedWith,OfficeStackAsyncWantToWorkWith,OfficeStackAsyncAdmired,OfficeStackSyncHaveWorkedWith,OfficeStackSyncWantToWorkWith,OfficeStackSyncAdmired,AISearchDevHaveWorkedWith,AISearchDevWantToWorkWith,AISearchDevAdmired,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOHow,SOComm,AISelect,AISent,AIBen,AIAcc,AIComplex,AIToolCurrently Using,AIToolInterested in Using,AIToolNot interested in Using,AINextMuch more integrated,AINextNo change,AINextMore integrated,AINextLess integrated,AINextMuch less integrated,AIThreat,AIEthics,AIChallenges,TBranch,ICorPM,WorkExp,Knowledge_1,Knowledge_2,Knowledge_3,Knowledge_4,Knowledge_5,Knowledge_6,Knowledge_7,Knowledge_8,Knowledge_9,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Frustration,ProfessionalTech,ProfessionalCloud,ProfessionalQuestion,Industry,JobSatPoints_1,JobSatPoints_4,JobSatPoints_5,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,SurveyLength,SurveyEase,JobSat
0,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,I don’t code outside of work,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",On the job training;Other online resources (e....,Technical documentation;Books;Written Tutorial...,API document(s) and/or SDK document(s);Traditi...,14,10,DevOps specialist,500 to 999 employees,I have some influence,Start a free trial;Ask developers I know/work ...,Is ready-to-go but also customizable for growt...,Quality support system;Reputation for quality ...,Spain,EUR European Euro,80000.0,Bash/Shell (all shells);Python;SQL,Go,NaN,NaN,NaN,NaN,OpenStack,NaN,NaN,Django;Flask,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APT;Docker;Kubernetes;Pip;Terraform,Kubernetes,Kubernetes,Vim;Visual Studio Code,NaN,NaN,Android;Ubuntu,Ubuntu,GitHub Discussions;Jira,NaN,NaN,Google Meet;Matrix;Mattermost,NaN,NaN,NaN,NaN,NaN,Stack Overflow;Stack Exchange,A few times per month or weekly,Yes,Less than once per month or monthly,Quickly finding code solutions,"Yes, somewhat","No, but I plan to soon",Favorable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Circulating misinformation or disinformation;M...,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN
1,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,I don’t code outside of work,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;On the job training;Oth...,Technical documentation;Blogs;Books;Stack Over...,API document(s) and/or SDK document(s);User gu...,15,11,Engineering manager,10 to 19 employees,I have a great deal of influence,Start a free trial;Ask developers I know/work ...,Is ready-to-go but also customizable for growt...,APIs;Customization;Quality support system;Repu...,Spain,EUR European Euro,85000.0,Bash/Shell (all shells);HTML/CSS;JavaScript;Py...,Bash/Shell (all shells);Go;HTML/CSS;JavaScript...,Bash/Shell (all shells);HTML/CSS;JavaScript;Py...,MariaDB;MongoDB;MySQL;SQLite,Cockroachdb;MariaDB;MySQL;PostgreSQL;SQLite,MariaDB;MySQL;SQLite,Amazon Web Services (AWS);Cloudflare;Google Cl...,Amazon Web Services (AWS);Cloudflare;Google Cl...,Amazon Web Ser

### Empiezo a trabajar columna por columna con los ecoders. 
0. MainBranch: Que tipo de programador eres:
- I am a developer by profession
- I am not primarily a developer, but I write code sometimes as part of my work/studies 
- I used to be a developer by profession, but no longer am 
- I am learning to code 
- I code primarily as a hobby 
- None of these

Decido hacer un Label encoder, con valores del 0 (None of these) al 5 (Developer by profession), ya que quiero que se le de eventualmente mas peso a estos perfiles

In [26]:
df['MainBranch'].unique()

array(['I am a developer by profession',
       'I am not primarily a developer, but I write code sometimes as part of my work/studies'],
      dtype=object)

In [27]:
labels = {#'None of these': 0, 
        #   'I code primarily as a hobby': 1, 
        #   'I am learning to code':2,  --------------> todos estos son valores que aparecen en la encuesta, pero no en las respuestas
        #   'I used to be a developer by profession, but no longer am':3, 
          'I am not primarily a developer, but I write code sometimes as part of my work/studies': 4,
          'I am a developer by profession':5}
df['MainBranch'] = df['MainBranch'].map(labels).fillna(-1)

In [28]:
df['MainBranch'].isna().sum()

0

1. Age:
- Under 18 years old 
- 18-24 years old 
- 25-34 years old 
- 35-44 years old 
- 45-54 years old 
- 55-64 years old 
- 65 years or older 
- Prefer not to say

Decido hacer un OneHotEncoder: aunque se agregarian mas columnas, no son muchas. No quiero que se le de mas importancia a unas edades que a otras. 

Antes de hacer el OneHotEncoder verifico que no hayan nans. Si los hay, los trato primero, y leugo hago OneHot

In [29]:
print(df['Age'].isna().sum())
print(df['Age'].dtype)
print(df['Age'].unique())
df['Age'] = df['Age'].fillna('Prefer not to say')  # Rellenar NaN con una categoría válida
df['Age'] = df['Age'].str.strip()   # por las dudas, elimino espacios vacios que me puedan causar problemas con el One Hot

0
object
['25-34 years old' '35-44 years old' '45-54 years old' '18-24 years old'
 '55-64 years old' '65 years or older' 'Under 18 years old'
 'Prefer not to say']


In [30]:
encoder = OneHotEncoder(sparse_output=False)

# Aplicamos el OneHotEncoder a la columna 'Age'
encoded = encoder.fit_transform(df[['Age']])

# Convertimos la matriz de salida a un DataFrame
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['Age']), index=df.index)

# Unimos el DataFrame original con las nuevas columnas codificadas
df = pd.concat([df, encoded_df], axis=1)

In [31]:
df = df.drop('Age', axis=1)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Columns: 118 entries, MainBranch to Age_Under 18 years old
dtypes: float64(20), int64(1), object(97)
memory usage: 722.9+ KB


Compruebo que todo esta correcto, se han agregado las columnas correspondientes, y las filas siguen igual. Sigo. 

2. Employment:
- Employed, full-time 
- Employed, part-time 
- Independent contractor, freelancer, or self-employed
- Not employed, but looking for work 
- Not employed, and not looking for work 
- Student, full-time 
- Student, part-time 
- Retired 
- I prefer not to say

Decido usar un LabelEncoder y darle una mayor importancia al empleado full-time.

In [33]:
df['Employment'].unique()

array(['Employed, full-time',
       'Employed, full-time;Independent contractor, freelancer, or self-employed',
       'Independent contractor, freelancer, or self-employed',
       'Employed, full-time;Student, part-time',
       'Student, part-time;Employed, part-time',
       'Student, full-time;Employed, part-time',
       'Employed, full-time;Independent contractor, freelancer, or self-employed;Student, part-time',
       'Not employed, but looking for work;Independent contractor, freelancer, or self-employed',
       'Employed, full-time;Not employed, but looking for work',
       'Employed, part-time',
       'Employed, full-time;Student, full-time;Independent contractor, freelancer, or self-employed',
       'Independent contractor, freelancer, or self-employed;Employed, part-time',
       'Employed, full-time;Independent contractor, freelancer, or self-employed;Employed, part-time',
       'Employed, full-time;Student, full-time',
       'Employed, full-time;Employed, part-ti

In [34]:
# Nuevo diccionario de mapeo
mapping = {
    'I prefer not to say': -1,
    'Retired': 0,
    'Not employed, and not looking for work': 0,
    'Not employed, but looking for work': 0,
    'Student, part-time': 0.5,
    'Student, full-time': 0,
    'Independent contractor, freelancer, or self-employed': 1,
    'Employed, part-time': 0.5,
    'Employed, full-time': 1
}

# Llenar los NaN con un valor temporal
df['Employment'].fillna('I prefer not to say', inplace=True)

# Función para asignar el valor mapeado
def map_employment(value):
    total_value = 0
    for key, val in mapping.items():
        if key in value:
            total_value += val
    return total_value

# Aplicar la función de mapeo
df['employment_map'] = df['Employment'].apply(map_employment)
df = df.drop(['Employment'], axis=1)

C:\Users\emita\AppData\Local\Temp\ipykernel_23696\846225123.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Employment'].fillna('I prefer not to say', inplace=True)


In [35]:
df['employment_map'].value_counts()

employment_map
1.0    691
2.0     49
1.5     31
0.5      8
2.5      5
Name: count, dtype: int64

3. RemoteWork:
- Remote 
- In-person 
- Hybrid (some remote, some in-person)

Decido un HoeHot (no son muchas columas, y quiero a todas con igualdad de importancia)

In [36]:
df['RemoteWork'].isna().sum()

0

In [37]:
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df[['RemoteWork']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['RemoteWork']), index=df.index)
df = pd.concat([df, encoded_df], axis=1)

In [38]:
df = df.drop('RemoteWork', axis=1)

4. CodingActivities (fuera del trabajo):
- Hobby (2)
- Freelance/contract work (3)
- Contribute to open-source projects (4) 
- Bootstrapping a business (5)
- School or academic work (1)
- Professional development or self-paced learning from online courses (6)
- I don’t code outside of work (0)
- Other (-1)

Decido un Label encoder y darle mas peso a unas categorias, mas que a otras (indicado el peso de cada una)

Hay Muchos Nan, los trabajo imputandolos a la categoria de "Other"

In [39]:
df['CodingActivities'].isna().sum()

0

In [40]:
df['CodingActivities'] = df['CodingActivities'].fillna('Other')

In [41]:
labels4 = {
    'Hobby': 2,
    'Freelance/contract work': 3,
    'Contribute to open-source projects': 4,
    'Bootstrapping a business': 5,
    'School or academic work':1,
    'Professional development or self-paced learning from online courses': 6,
    'I don’t code outside of work': 0,
    'Other': -1
}
df['CodingActivities'] = df['CodingActivities'].map(labels4).fillna(-1)

In [42]:
df['CodingActivities'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 784 entries, 0 to 783
Series name: CodingActivities
Non-Null Count  Dtype  
--------------  -----  
784 non-null    float64
dtypes: float64(1)
memory usage: 6.2 KB


5. EdLevel: 
- Primary/elementary school (0)
- Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.) (1)
- Some college/university study without earning a degree (2)
- Associate degree (A.A., A.S., etc.) (3)
- Bachelor’s degree (B.A., B.S., B.Eng., etc.) (4) 
- Master’s degree (M.A., M.S., M.Eng., MBA, etc.) (5) 
- Professional degree (JD, MD, Ph.D, Ed.D, etc.) (6)
- Something else (-1)

Hago un LabelEncoder con los valores destacados

In [43]:
df['EdLevel'].unique()

array(['Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
       'Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
       'Professional degree (JD, MD, Ph.D, Ed.D, etc.)',
       'Some college/university study without earning a degree',
       'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
       'Associate degree (A.A., A.S., etc.)', 'Something else',
       'Primary/elementary school'], dtype=object)

In [44]:
df['EdLevel'].isna().sum()

0

In [45]:
labels5 = {
    'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 5,
    'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 4,
    'Professional degree (JD, MD, Ph.D, Ed.D, etc.)': 6,
    'Some college/university study without earning a degree': 2, 
    'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 1,
    'Associate degree (A.A., A.S., etc.)': 3, 
    'Something else': -1,
    'Primary/elementary school': 0
}
df['EdLevel'] = df['EdLevel'].map(labels5).fillna(-1)

6. LearnCode: 
- Books / Physical media (3): Implica un esfuerzo autodirigido, pero es formal y metódico.
- Coding Bootcamp (3): También estructurado, pero más intensivo y con un enfoque en habilidades prácticas.
- Colleague (2): Más relacionado con la práctica laboral, pero también limitado al entorno de trabajo.
- Friend or family member (1): Aprendizaje menos formal y probablemente limitado en alcance.
- Online Courses or Certification (2): Similar al entrenamiento en el trabajo, pero autodirigido y menos formal.
- On the job training (2): Aprender en el trabajo es práctico, pero menos estructurado.
- Other online resources (e.g., videos, blogs, forum, online community) (2): Aunque útil, suele ser más informal y variado en calidad.
- School (i.e., University, College, etc) (4): Representa una forma tradicional de aprendizaje, estructurada y generalmente reconocida como valiosa.
- Other (please specify):(0)

Le diversa importancia a las categorias con total discrecion (segun el tipo de educacion). Un OneHot me crearia demasiadas columnas. 

Prefiero hacer una funcion que, luego de dividir en el ";", se reemplace cada categoria por el numero correspondiente, y luego sumo esos numeros. De esta forma, una persona que se ha educado de 5 fuentes diversas, tendra mas puntaje que una que solo se ha educado con 1. Da cuenta del compromiso y profesionalismo que tiene esa persona. 

In [46]:
df['LearnCode'].isna().sum()   # hay un solo valor, lo dejamos a 0 (primera linea de la funcion)

1

Creo en un dict los valores

In [47]:
importance_dict = {
    'Books / Physical media': 3,
    'Coding Bootcamp': 3,
    'Colleague': 2,
    'Friend or family member': 1,
    'Online Courses or Certification': 2,
    'On the job training': 2,
    'Other online resources (e.g., videos, blogs, forum, online community)': 2,
    'School (i.e., University, College, etc)': 4,
    'Other (please specify):': 0
}

Creo la funcion que lo divida en el ; y se le asigne a cada categoria un valor. Luego se suman todos los valores

In [48]:
def calculate_score(x):
    # Si el valor es NaN, devolvemos 0
    if pd.isna(x):
        return 0
    
    # Dividimos el texto en una lista de valores, usando ';' como separador
    values = x.split(';')
    
    # Inicializamos el puntaje en 0
    total_score = 0
    
    # Iteramos por cada valor en la lista
    for value in values:
        # Quitamos espacios en blanco alrededor del texto
        cleaned_value = value.strip()
        
        # Obtenemos el puntaje del diccionario y lo sumamos al puntaje total
        total_score += importance_dict.get(cleaned_value, 0)
    
    # Devolvemos el puntaje total
    return total_score

In [49]:
df['LearnCode'] = df['LearnCode'].apply(calculate_score)

7. LearnCodeOnline
- Stack Overflow ('Community_Collaboration')
- Blogs ('Community_Collaboration')
- Online community (including social media) ('Community_Collaboration')
- Video-based online courses ('Visual_Interactive')
- Online challenges (e.g., daily or weekly coding challenges) ('Community_Collaboration')
- Coding sessions (live or recorded) ('Community_Collaboration')
- Written-based online courses ('Technical_Reference')
- How-to videos ('Visual_Interactive')
- Auditory material (e.g., podcasts) ('Visual_Interactive')
- Technical documentation ('Technical_Reference')
- Certification videos ('Visual_Interactive')
- Books ('Technical_Reference')
- Games that teach programming ('Visual_Interactive')
- AI chatbot or other tool ('Community_Collaboration')
- Written tutorials ('Technical_Reference')
- Interactive tutorial ('Visual_Interactive')
- Other ('Visual_Interactive')

Estas herramientas no tienen mas importancia unas que otras, ni son ordinales. Elijo usar OneHotEncoder. Como son muchas columnas, decido agruparlas, para que no sean tantas.
Dejo 3 categorias: 'Visual_Interactive', 'Technical_Reference', 'Community_Collaboration'. 

In [50]:
df['LearnCodeOnline'].unique()

array(['Technical documentation;Books;Written Tutorials;Stack Overflow;Written-based Online Courses',
       'Technical documentation;Blogs;Books;Stack Overflow;Coding sessions (live or recorded);Interactive tutorial;Video-based Online Courses',
       'Technical documentation;Blogs;Books;Written Tutorials;Stack Overflow;Social Media;How-to videos;Interactive tutorial;Video-based Online Courses;Written-based Online Courses;Online challenges (e.g., daily or weekly coding challenges);Programming Games',
       'Technical documentation;Books;Video-based Online Courses', nan,
       'Technical documentation;Written Tutorials;How-to videos',
       'Technical documentation;Stack Overflow;How-to videos;AI',
       'Technical documentation;Books;Written Tutorials;Stack Overflow;Coding sessions (live or recorded);AI;Online challenges (e.g., daily or weekly coding challenges)',
       'Technical documentation;Blogs;Books;Written Tutorials;Stack Overflow;Coding sessions (live or recorded);Social

Creo una funcion para ver los valores unicos, ya que en las celdas hay muchos valores. 

In [51]:
def get_unique_values(df, column_name):
    # Inicializamos un conjunto vacío para almacenar valores únicos
    unique_values = set()

    # Iteramos sobre cada fila en la columna
    for value in df[column_name]:
        # Si el valor no es nulo (NaN)
        if pd.notna(value):
            # Si el valor contiene múltiples elementos separados por ';'
            if isinstance(value, str) and ';' in value:
                # Dividimos el string y añadimos cada elemento al conjunto
                unique_values.update(value.split(';'))
            else:
                # Si es un valor único, lo añadimos directamente al conjunto
                unique_values.add(value)

    # Convertimos el conjunto a una lista y la ordenamos
    return sorted(list(unique_values))

In [52]:
unique_list = get_unique_values(df, 'LearnCodeOnline')

unique_list

['AI',
 'Auditory material (e.g., podcasts)',
 'Blogs',
 'Books',
 'Certification videos',
 'Coding sessions (live or recorded)',
 'How-to videos',
 'Interactive tutorial',
 'Online challenges (e.g., daily or weekly coding challenges)',
 'Other (Please specify):',
 'Programming Games',
 'Social Media',
 'Stack Overflow',
 'Technical documentation',
 'Video-based Online Courses',
 'Written Tutorials',
 'Written-based Online Courses']

Mapeo las categorias para que no sean tantas al hacer OneHot

In [53]:
# Primero, creamos un diccionario para mapear las herramientas a las categorías
category_map = {
    'Stack Overflow': 'Community_Collaboration',
    'Blogs': 'Community_Collaboration',
    'Social Media': 'Community_Collaboration',
    'Video-based online courses': 'Visual_Interactive',
    'Online challenges (e.g., daily or weekly coding challenges)': 'Community_Collaboration',
    'Coding sessions (live or recorded)': 'Community_Collaboration',
    'Written-based online courses': 'Technical_Reference',
    'How-to videos': 'Visual_Interactive',
    'Auditory material (e.g., podcasts)': 'Visual_Interactive',
    'Technical documentation': 'Technical_Reference',
    'Certification videos': 'Visual_Interactive',
    'Books': 'Technical_Reference',
    'Games': 'Visual_Interactive',
    'AI': 'Community_Collaboration',
    'Written tutorials': 'Technical_Reference',
    'Interactive tutorial': 'Visual_Interactive',
    'Other (Please specify):': 'Visual_Interactive'
}
# Lleno los Nan con 'Other'
df['LearnCodeOnline'] = df['LearnCodeOnline'].fillna('Other (Please specify):')
# Aplicamos el mapeo a la columna 'LearnCodeOnline'
df['LearnCodeOnline_Category'] = df['LearnCodeOnline'].map(category_map)

In [54]:
# Creamos el OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Aplicamos el OneHotEncoder
encoded = encoder.fit_transform(df[['LearnCodeOnline_Category']])

# Creamos un DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['LearnCodeOnline_Category']), index=df.index)

# Unimos el DataFrame original con las nuevas columnas codificadas
df = pd.concat([df, encoded_df], axis=1)

# Eliminamos la columna temporal 'LearnCodeOnline_Category' si ya no la necesitas
df = df.drop(['LearnCodeOnline_Category', 'LearnCodeOnline'], axis=1)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Columns: 122 entries, MainBranch to LearnCodeOnline_Category_nan
dtypes: float64(28), int64(3), object(91)
memory usage: 747.4+ KB


8. TechDoc
- API document(s) and/or SDK document(s) 
- User guides or README files found in the source repository 
- First-party knowledge base 
- Traditional public search engine
- AI-powered search/dev tool (free) 
- AI-powered search/dev tool (paid)
- Other 

Separar por comas y OneHotEncoder  (lo hago en el codigo del final, con el resto de las columnas donde hago esto.)
Los Nan los dejo asi, para que simplemente tenga 0 en todas las columnas creadas con el OneHot. 

In [56]:
df['TechDoc'].isnull().sum()

262

In [57]:
unique_list = get_unique_values(df, 'TechDoc')

# Imprimimos la lista de valores únicos
unique_list

['AI-powered search/dev tool (free)',
 'AI-powered search/dev tool (paid)',
 'API document(s) and/or SDK document(s)',
 'First-party knowledge base',
 'Other (please specify):',
 'Traditional public search engine',
 'User guides or README files found in the source repository']

9. YearsCode: sacar los nan y pasarlo a int

In [58]:
df['YearsCode'].isnull().sum()
moda = df['YearsCode'].mode()[0]
df['YearsCode'] = df['YearsCode'].fillna(moda)

In [59]:
df['YearsCode']=df['YearsCode'].astype(int)

In [60]:
df['YearsCode'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 784 entries, 0 to 783
Series name: YearsCode
Non-Null Count  Dtype
--------------  -----
784 non-null    int32
dtypes: int32(1)
memory usage: 3.2 KB


10. YearsCodePro: reemplazo nan y paso a int

In [61]:
df['YearsCodePro'].isnull().sum()

5

In [62]:
moda = df['YearsCodePro'].mode()[0]
df['YearsCodePro'] = df['YearsCodePro'].fillna(moda)

In [63]:
df['YearsCodePro'].unique()

array(['10', '11', '24', '22', '31', '14', '2', '7', '21', '28', '6', '4',
       '16', '8', '9', '1', '26', '12', '20', '23', '13', '3', '25', '5',
       '18', '33', '32', 'Less than 1 year', '15', '27', '17', '30', '19',
       '29', '38', '44', '39', '37', '45', '34', '35'], dtype=object)

In [64]:
df['YearsCodePro']=df['YearsCodePro'].replace('Less than 1 year', 0)

In [65]:
df['YearsCodePro']=df['YearsCodePro'].astype(int)
df['YearsCodePro'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 784 entries, 0 to 783
Series name: YearsCodePro
Non-Null Count  Dtype
--------------  -----
784 non-null    int32
dtypes: int32(1)
memory usage: 3.2 KB


11. DevType
- Academic researcher (5)
- Blockchain (6)
- Cloud infrastructure engineer (3)
- Data or business analyst (2)
- Data engineer (2)
- Data scientist or machine learning specialist (2)
- Database administrator (7)
- Designer (7)
- Developer Advocate (1)
- Developer, AI (1)
- Developer, back-end (1)
- Developer, desktop or enterprise applications (1)
- Developer, embedded applications or devices (1)
- Developer Experience (1)
- Developer, front-end (1)
- Developer, full-stack(1)
- Developer, game or graphics (1)
- Developer, mobile(1) 
- Developer, QA or test (1)
- DevOps specialist (3)
- Educator (5)
- Engineer, site reliability (3)
- Engineering manager (4)
- Hardware Engineer (6)
- Marketing or sales professional (7)
- Product manager (4)
- Project manager (4)
- Research & Development role (5)
- Scientist (5)
- Senior Executive (C-Suite, VP, etc.) (4)
- Student (7)
- System administrator (3)
- Security professional (3)
- Other (7)

Elijo agruparlos para que no sean tantos, y asi simplificar el analisis, y luego un OneHotEncoder:
1. Gestión y Liderazgo
2. Desarrollo de Software
3. Infraestructura y Sistemas
4. Ingeniería de Datos y Ciencia de Datos
5. Investigación y Educación
6. Hardware y Blockchain
7. Roles de Soporte y Otros

In [66]:
# Primero me fijo que no hayan Nan
df['DevType'].isnull().sum()

4

In [67]:
df['DevType']=df['DevType'].fillna('Other (please specify):')

In [68]:
df['DevType'].unique()

array(['DevOps specialist', 'Engineering manager',
       'Developer, full-stack', 'Academic researcher',
       'Other (please specify):', 'Developer, back-end',
       'Senior Executive (C-Suite, VP, etc.)', 'Developer, front-end',
       'Developer Advocate',
       'Developer, desktop or enterprise applications',
       'Developer, mobile', 'Research & Development role',
       'Data scientist or machine learning specialist',
       'Developer, embedded applications or devices',
       'Cloud infrastructure engineer', 'System administrator',
       'Data engineer', 'Developer Experience', 'Developer, QA or test',
       'Designer', 'Developer, AI', 'Engineer, site reliability',
       'Data or business analyst', 'Developer, game or graphics',
       'Security professional', 'Scientist', 'Educator',
       'Product manager', 'Project manager', 'Hardware Engineer',
       'Blockchain'], dtype=object)

In [69]:
# Primero, creamos un diccionario para mapear las herramientas a las categorías
dev_type_map = {
    'Academic researcher': 'Investigación y Educación',
    'Blockchain': 'Hardware y Blockchain',
    'Cloud infrastructure engineer': 'Infraestructura y Sistemas',
    'Data engineer': 'Ingeniería de Datos y Ciencia de Datos',
    'Data or business analyst': 'Ingeniería de Datos y Ciencia de Datos',
    'Data scientist or machine learning specialist': 'Ingeniería de Datos y Ciencia de Datos',
    'Designer': 'Roles de Soporte y Otros',
    'DevOps specialist': 'Infraestructura y Sistemas',
    'Developer Advocate': 'Desarrollo de Software',
    'Developer Experience': 'Desarrollo de Software',
    'Developer, AI': 'Desarrollo de Software',
    'Developer, QA or test': 'Desarrollo de Software',
    'Developer, back-end': 'Desarrollo de Software',
    'Developer, desktop or enterprise applications': 'Desarrollo de Software',
    'Developer, embedded applications or devices': 'Desarrollo de Software',
    'Developer, front-end': 'Desarrollo de Software',
    'Developer, full-stack': 'Desarrollo de Software',
    'Developer, game or graphics': 'Desarrollo de Software',
    'Developer, mobile': 'Desarrollo de Software',
    'Educator': 'Investigación y Educación',
    'Engineer, site reliability': 'Infraestructura y Sistemas',
    'Engineering manager': 'Gestión y Liderazgo',
    'Hardware Engineer': 'Hardware y Blockchain',
    'Other (please specify):': 'Roles de Soporte y Otros',
    'Product manager': 'Gestión y Liderazgo',
    'Project manager': 'Gestión y Liderazgo',
    'Research & Development role': 'Investigación y Educación',
    'Scientist': 'Investigación y Educación',
    'Security professional': 'Infraestructura y Sistemas',
    'Senior Executive (C-Suite, VP, etc.)': 'Gestión y Liderazgo',
    'System administrator': 'Infraestructura y Sistemas',
    # 'Marketing or sales professional': 'Roles de Soporte y Otros',
    # 'Student': 'Roles de Soporte y Otros',
    # 'Database administrator': 'Roles de Soporte y Otros'    
}
len(dev_type_map)

31

In [70]:
# Aplicamos el mapeo a la columna 'DevType'
df['DevType_Category'] = df['DevType'].map(dev_type_map)

# Creamos el OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Aplicamos el OneHotEncoder
encoded = encoder.fit_transform(df[['DevType_Category']])

# Creamos un DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['DevType_Category']), index=df.index)

# Unimos el DataFrame original con las nuevas columnas codificadas
df = pd.concat([df, encoded_df], axis=1)

# Eliminamos la columna temporal 'DevType_Category' si ya no la necesitas
df = df.drop(['DevType_Category', 'DevType'], axis=1)

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Columns: 128 entries, MainBranch to DevType_Category_Roles de Soporte y Otros
dtypes: float64(35), int32(2), int64(3), object(88)
memory usage: 778.0+ KB


12. OrgSize: Cuántas personas forman parte de la organización para la que trabajas (organización primaria)
- I don’t know
- Just me - I am a freelancer, sole proprietor, etc. 
- 2 to 9 employees 
- 10 to 19 employees 
- 20 to 99 employees 
- 100 to 499 employees 
- 500 to 999 employees 
- 1000 to 4999 employees 
- 5000 to 9999 employees 
- 10000 or more employees 

Label Encoder, en ese orden

In [72]:
df['OrgSize'].unique()

array(['500 to 999 employees', '10 to 19 employees', '20 to 99 employees',
       '100 to 499 employees', '5,000 to 9,999 employees',
       '1,000 to 4,999 employees', '2 to 9 employees',
       '10,000 or more employees',
       'Just me - I am a freelancer, sole proprietor, etc.',
       'I don’t know', nan], dtype=object)

In [73]:
# Primero trato los Nan
df['OrgSize'] = df['OrgSize'].fillna('I don’t know')
print(df['OrgSize'].isna().sum())

0


In [74]:
type(df.iloc[4, 8])

str

Los valores NaN siguen apareciendo, a pesar de hacer utilizado el fillna mas arriba. Por eso lo pongo tambien al hacer el map. 

In [75]:
labels12 = {
    'I don’t know': -1,
    'Just me - I am a freelancer, sole proprietor, etc.': 0,
    '2 to 9 employees': 1,
    '10 to 19 employees': 2,
    '20 to 99 employees': 3,
    '100 to 499 employees': 4,
    '500 to 999 employees': 5, 
    '1000 to 4999 employees': 6, 
    '5000 to 9999 employees': 7,
    '10000 or more employees': 8 
}
df['OrgSize'] = df['OrgSize'].map(labels12).fillna(-1)

In [76]:
# Otra opcion es forzar la conversion con este codigo: # Forzar la conversión a string antes de llenar
# df['OrgSize'] = df['OrgSize'].astype(str).replace('nan', np.nan).fillna("I don't know")

13. PurchaseInfluence: Nivel de influencia sobre nuevas tecnologías adquiridas por la organización
- I have little or no influence 
- I have some influence 
- I have a great deal of influence

Label Encoder en ese orden

In [77]:
# Primero trato los Nan
df['PurchaseInfluence'] = df['PurchaseInfluence'].fillna('Other')
print(df['PurchaseInfluence'].isna().sum())

0


In [78]:
labels13 = {
    'I have little or no influence': 0,
    'I have some influence': 1,
    'I have a great deal of influence': 2,
    'Other': -1
}
df['PurchaseInfluence'] = df['PurchaseInfluence'].map(labels13).fillna(-1)

14. BuyNewTool: Cómo descubrir o investigas soluciones disponibles en nuevas herramientas o softwares
- Start a free trial 
- Ask developers I know/work with 
- Ask a generative AI tool 
- Visit developer communities like Stack Overflow 
- Read ratings or reviews on third party sites like G2 Crowd 
- Research companies that have advertised on sites I visit 
- Research companies that have emailed me
- Other 

No tienen orden, separo por ; y hago OneHotEncoder al final

In [79]:
df['BuyNewTool'].isna().sum()
df['BuyNewTool'] = df['BuyNewTool'].fillna('Other')

15. BuildvsBuy: Irrelevante, la elimino. Además, no está en el df 2023. 

In [80]:
df = df.drop(['BuildvsBuy'], axis=1)

16. TechEndorse: Que caracteristica crees más importante a la hora de adquirir una nueva tecnología
- APIs 
- Customization
- Integrated developer community 
- Connected to an open source project 
- Quality support system AI tool integration 
- Reputation for quality and excellence 
- Frequent updates to features
- Other

Divido por ; u hago OneHotEncoder. Todo al final 

In [81]:
df['TechEndorse']

0      Quality support system;Reputation for quality ...
1      APIs;Customization;Quality support system;Repu...
2                                                    NaN
3                  Reputation for quality and excellence
4      Quality support system;Connected to an open so...
                             ...                        
779                       Integrated developer community
780                                                  NaN
781    APIs;Customization;Integrated developer commun...
782    APIs;Customization;Integrated developer commun...
783                                                  NaN
Name: TechEndorse, Length: 784, dtype: object

In [82]:
# Primero trato los Nan
df['TechEndorse'].isna().sum()
df['TechEndorse'] = df['TechEndorse'].fillna('Other')

In [83]:
unique_list = get_unique_values(df, 'TechEndorse')

unique_list

['AI tool integration',
 'APIs',
 'Connected to an open source project',
 'Customization',
 'Frequent updates to features',
 'Integrated developer community',
 'Other',
 'Other:',
 'Quality support system',
 'Reputation for quality and excellence']

17. Country:
Si hago solo de Spain, la elimino. 

18. Currency: Si hago solo de Europa o Spain, la elimino 

In [84]:
df = df.drop(['Country', 'Currency'], axis=1)

#### Ahora vienen una seguidilla de columnas que tengo que separar por el ';' y luego hacer un OneHotEncoder. 

Creo una funcion para aplicar lo mismo a todas juntas. El codigo de estas columnas se vera al final.

Luego elimino las que indico en el listado, por no aportar informacion nueva ni relevante

19. LanguageHaveWorkedWith: Dividirlo por ';' y luego OneHotEncoder. Se haran muchas columnas, pero no queda otra. 

20. LanguageWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

21. LanguageAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

22. DatabaseHaveWorkedWith: Igual

23. DatabaseWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

24. DatabaseAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

25. PlatformHaveWorkedWith: Igual

26. PlatformWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

27. PlatformAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

28. WebframeHaveWorkedWith: Igual

29. WebframeWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

30. WebframeAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

31. EmbeddedHaveWorkedWith: Igual 

32. EmbeddedWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas 

33. EmbeddedAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas 

34. MiscTechHaveWorkedWith: Igual 

35. MiscTechWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas 

36. MiscTechAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas 

37. ToolsTechHaveWorkedWith: Igual 

38. ToolsTechWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

39. ToolsTechAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

40. NEWCollabToolsHaveWorkedWith: Igual 

41. NEWCollabToolsWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

42. NEWCollabToolsAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

43. OpSysPersonal: La elimino, me quedo solo con el uso de sistema operativo profesional. 

44. OpSysProfessional use: Agrupo los sistemas operativos en los mas representativos para no hacer tantas columnas: 
- AIX (4) Otros
- Android (3) Android-based
- Arch (1)
- BSD (1)
- ChromeOS (3)
- Cygwin (4)
- Debian (1) Linux-based
- Fedora (1)
- Haiku (1)
- iOS (2) MacOs
- iPadOS (2)
- MacOS (2)
- Other Linux-based (1)
- Red Hat (1)
- Solaris (1)
- Ubuntu (1)
- Windows (0) Windows-based
- Windows Subsystem for Linux (WSL) (0)
- Other (4)

Una vez agrupadas, le hago un MultilabelEncoder para poder poner varias etiquetas por fila

In [85]:
# Primero trato los Nan
df['OpSysProfessional use'] = df['OpSysProfessional use'].fillna('Other')
df['OpSysProfessional use'].isnull().sum()

0

In [86]:
# Diccionario de mapeo
os_map = {
    'AIX': 'Otros',
    'Android': 'Android-based',
    'Arch': 'Linux-based',
    'BSD': 'Linux-based',
    'ChromeOS': 'Android-based',
    'Cygwin': 'Otros',
    'Debian': 'Linux-based',
    'Fedora': 'Linux-based',
    'Haiku': 'Linux-based',
    'iOS': 'MacOS',
    'iPadOS': 'MacOS',
    'MacOS': 'MacOS',
    'Other Linux-based': 'Linux-based',
    'Red Hat': 'Linux-based',
    'Solaris': 'Linux-based',
    'Ubuntu': 'Linux-based',
    'Windows': 'Windows-based',
    'Windows Subsystem for Linux (WSL)': 'Windows-based',
    'Other': 'Otros'
}

In [87]:
# Función modificada para manejar NaN
def split_and_map(value):
    if pd.isna(value):
        return []  # Retorna una lista vacía para NaN
    return [os_map.get(os.strip(), 'Otros') for os in value.split(';')]

# Aplicar la función a la columna
df['OpSysProfessional_Category'] = df['OpSysProfessional use'].apply(split_and_map)

# Usar MultiLabelBinarizer para el one-hot encoding
mlb = MultiLabelBinarizer()
encoded = mlb.fit_transform(df['OpSysProfessional_Category'])

# Crear DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded, columns=mlb.classes_, index=df.index)

# Unir el DataFrame original con las nuevas columnas codificadas
df = pd.concat([df, encoded_df], axis=1)

# Eliminar las columnas originales si ya no las necesitas
df = df.drop(['OpSysProfessional_Category', 'OpSysProfessional use'], axis=1)

# Verificar que no se haya creado una columna 'nan'
if 'nan' in encoded_df.columns:
    df = df.drop('nan', axis=1)

45. OfficeStackAsyncHaveWorkedWith (Herramientas de documentación que has usado): Igual 

46. OfficeStackAsyncWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

47. OfficeStackAsyncAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

48. OfficeStackSyncHaveWorkedWith(Herramientas de comunicación que has usado): Igual

49. OfficeStackSyncWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

50. OfficeStackSyncAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas 

51. AISearchDevHaveWorkedWith: Igual

52. AISearchDevWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

53. AISearchDevAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

54. NEWSOSites(Uso de StackOverFlow): No tiene relevancia para mi analisis en concreto, lo elimino. 

55. SOVisitFreq: No tiene relevancia para mi analisis en concreto, lo elimino.

56. SOAccount: No tiene relevancia para mi analisis en concreto, lo elimino.

57. SOPartFreq: No tiene relevancia para mi analisis en concreto, lo elimino.

58. SOHow: No tiene relevancia para mi analisis en concreto, lo elimino.

59. SOComm: No tiene relevancia para mi analisis en concreto, lo elimino.

60. AISelect (Usas IA para tu proceso de desarrollo):
- Yes 
- No, but I plan to soon 
- No, and I don't plan to
Elijo Label encoder con (2) para el Yes.

In [88]:
df['AISelect'].unique()

array(['No, but I plan to soon', 'Yes', "No, and I don't plan to", nan],
      dtype=object)

In [89]:
# Primero trato los Nan
df['AISelect'] = df['AISelect'].fillna('Other')
print(df['AISelect'].isna().sum())

0


In [90]:
labels60 = {
    "No, and I don't plan to": 0,
    'No, but I plan to soon': 1,
    'Yes': 2,
    'Other': -1
}
df['AISelect'] = df['AISelect'].map(labels60).fillna(-1)

61. AISent (Uso de herramientas de IA como parte de su flujo de trabajo de desarrollo): 
- Very favorable (5) 
- Favorable(4)
- Indifferent (3)
- Unfavorable (2)
- Very unfavorable (1)
- Unsure (0)

LabelEncoder con los valores indicados. 

In [91]:
df['AISent'].unique()

array(['Favorable', 'Unsure', 'Indifferent', 'Very favorable', nan,
       'Unfavorable', 'Very unfavorable'], dtype=object)

In [92]:
# Primero trato los Nan
df['AISent'] = df['AISent'].fillna('Unsure')
print(df['AISent'].isna().sum())

0


In [93]:
labels61 = {
    'Very favorable': 5, 
    'Favorable': 4, 
    'Indifferent': 3, 
    'Unfavorable': 2,
    'Very unfavorable': 1,
    'Unsure': 0
}
df['AISent'] = df['AISent'].map(labels61).fillna(-1)

62. AIBen (beneficios importantes espera lograr con la IA): 
- Increase productivity 
- Greater efficiency 
- Make workload more manageable 
- Improve collaboration 
- Speed up learning 
- Improve accuracy in coding 
- None of the above 
- Other 

Dividirlo por ';' y luego OneHotEncoder. 

63. AIAcc (Cuanto confias en el output de la IA como parte de tu trabajo):
- Highly trust (4)
- Somewhat trust (3)
- Neither trust nor distrust (2) 
- Somewhat distrust (1)
- Highly distrust (0)

LabelEncoder con los valores indicados 

In [94]:
# Primero trato los Nan
df['AIAcc'] = df['AIAcc'].fillna('Other')
print(df['AIAcc'].isna().sum())

0


In [95]:
df['AIAcc'].unique()

array(['Other', 'Neither trust nor distrust', 'Somewhat trust',
       'Somewhat distrust', 'Highly trust', 'Highly distrust'],
      dtype=object)

In [96]:
labels63 = {
    'Highly trust': 4, 
    'Somewhat trust': 3, 
    'Neither trust nor distrust': 2,
    'Somewhat distrust': 1, 
    'Highly distrust': 0, 
    'Other': -1
}
df['AIAcc'] = df['AIAcc'].map(labels63).fillna(-1)

64. AIComplex (Que tan bien manejan tareas complejas las herramientas de IA que usas):
- Very well at handling complex tasks (4)
- Good, but not great at handling complex tasks (3)
- Neither good or bad at handling complex tasks (2)
- Bad at handling complex tasks (1)
- Very poor at handling complex tasks (0)

LabelEncoder, con los valores indicados 

In [97]:
# Primero trato los Nan
df['AIComplex'] = df['AIComplex'].fillna('Other')
print(df['AIComplex'].isna().sum())

0


In [98]:
df['AIComplex'].unique()

array(['Other', 'Neither good or bad at handling complex tasks',
       'Good, but not great at handling complex tasks',
       'Bad at handling complex tasks',
       'Very well at handling complex tasks',
       'Very poor at handling complex tasks'], dtype=object)

In [99]:
labels64 = {
    'Very well at handling complex tasks': 4, 
    'Good, but not great at handling complex tasks': 3, 
    'Neither good or bad at handling complex tasks': 2,
    'Bad at handling complex tasks': 1, 
    'Very poor at handling complex tasks': 0, 
    'Other': -1
}
df['AIComplex'] = df['AIComplex'].map(labels64).fillna(-1)

65. AIToolCurrently Using:
- Learning about a codebase 
- Project planning 
- Writing code 
- Documenting code 
- Debugging and getting help 
- Testing code 
- Committing and reviewing code 
- Deployment and monitoring 
- Predictive analytics 
- Search for answers 
- Generating content or synthetic data 
- Other 

Dividirlo por ';' y luego OneHotEncoder.

66. AIToolInterested in Using: La elimino, dejo solo la de "Currently in using" como representativa. Sino se me hacen muchas columnas y muy repetitivas

67. AIToolNot interested in Using: La elimino, dejo solo la de "Currently in using" como representativa. Sino se me hacen muchas columnas y muy repetitivas

68. AINextMuch more integrated (Cuales estarán mucho más integradas): 
- Learning about a codebase 
- Project planning 
- Writing code 
- Documenting code 
- Debugging and getting help 
- Testing code 
- Committing and reviewing code 
- Deployment and monitoring 
- Predictive analytics 
- Search for answers 
- Generating content or synthetic data 
- Other 

Dividirlo por ';' y luego OneHotEncoder. 

69. AINextNo change: La elimino, dejo solo la de "Much more integrated" como representativa. Sino se me hacen muchas columnas y muy repetitivas

70. AINextMore integrated: La elimino, dejo solo la de "Much more integrated" como representativa. Sino se me hacen muchas columnas y muy repetitivas

71. AINextLess integrated: La elimino, dejo solo la de "Much more integrated" como representativa. Sino se me hacen muchas columnas y muy repetitivas

72. AINextMuch less integrated: La elimino, dejo solo la de "Much more integrated" como representativa. Sino se me hacen muchas columnas y muy repetitivas

73. AIThreat: 
- Yes (2)
- No (0)
- I'm not sure (1)

LabelEncoder con los valores indicados 

In [100]:
# Primero trato los Nan
df['AIThreat'] = df['AIThreat'].fillna("I'm not sure")
print(df['AIThreat'].isna().sum())

0


In [101]:
df['AIThreat'].unique()

array(['No', 'Yes', "I'm not sure"], dtype=object)

In [102]:
labels73 = {
    'Yes': 2,
    'No': 0, 
    "I'm not sure": 1, 
}
df['AIThreat'] = df['AIThreat'].map(labels73).fillna(-1)

74. AIEthics (Qué responsabilidades éticas de la IA son importantes para ti): No creo que sea muy relevante, la elimino. 

75. AIChallenges (Qué desafíos enfrenta tu equipo/compañía a la hora de usar asistentes de código con IA o GenAi):
- Don’t trust the output or answers 
- AI tools lack context of codebase, internal architecture, and/or company knowledge 
- They create more work (more code/PRs to review, etc.) 
- We don’t have the right policies in place to reduce security risks 
- Not everyone uses them 
- Lack of executive buy-in 
- Lack of proper training and education on new tools 
- None of the above 
- Other 

Dividirlo por ';' y luego OneHotEncoder.

76. TBranch: Irrelevante, la elimino

77. ICorPM: Irrelevante, la elimino

78. WorkExp: Los dejo como estan 

79. Knowledge_1: Irrelevante, lo elimino

80. Knowledge_2: Irrelevante, lo elimino

81. Knowledge_3: Irrelevante, lo elimino

82. Knowledge_4: Irrelevante, lo elimino

83. Knowledge_5: Irrelevante, lo elimino

84. Knowledge_6: Irrelevante, lo elimino

85. Knowledge_7: Irrelevante, lo elimino

86. Knowledge_8: Irrelevante, lo elimino

87. Knowledge_9: Irrelevante, lo elimino

88. Frequency_1 (Qué tan frecuente experimentas ‘Needing help from people outside of your immediate team?‘): LabelEncoder 

In [103]:
# Primero trato los Nan
df['Frequency_1'] = df['Frequency_1'].fillna('Other')
print(df['Frequency_1'].isna().sum())

0


In [104]:
df['Frequency_1'].unique()

array(['Other', '1-2 times a week', '3-5 times a week', 'Never',
       '6-10 times a week', '10+ times a week'], dtype=object)

In [105]:
labels88 = {
    '10+ times a week': 4, 
    '6-10 times a week': 3, 
    '3-5 times a week': 2,
    '1-2 times a week': 1, 
    'Never': 0, 
    'Other': -1
}
df['Frequency_1'] = df['Frequency_1'].map(labels88).fillna(-1)

89. Frequency_2 (Qué tan frecuente experimentas ‘Interacting with people outside of your immediate team?’):  LabelEncoder

Mismas categorias que la 88, uso ese labels

In [106]:
df['Frequency_2'].unique()

array([nan, '3-5 times a week', '1-2 times a week', 'Never',
       '6-10 times a week', '10+ times a week'], dtype=object)

In [107]:
# Primero trato los Nan
df['Frequency_2'] = df['Frequency_2'].fillna('Other')
print(df['Frequency_2'].isna().sum())

0


In [108]:
df['Frequency_2'] = df['Frequency_2'].map(labels88).fillna(-1)

90. Frequency_3 (Qué tan frecuente experimentas ‘Encountering knowledge silos (where one individual or team has information that's not shared or distributed with other individuals or teams) at work?’): LabelEncoder

Mismas categorias que la 88, uso ese labels

In [109]:
df['Frequency_3'].unique()

array([nan, '1-2 times a week', 'Never', '10+ times a week',
       '3-5 times a week', '6-10 times a week'], dtype=object)

In [110]:
# Primero trato los Nan
df['Frequency_3'] = df['Frequency_3'].fillna('Other')
print(df['Frequency_3'].isna().sum())

0


In [111]:
df['Frequency_3'] = df['Frequency_3'].map(labels88).fillna(-1)

91. TimeSearching (Promedio de tiempo que gastas buscando respuesta o solución a algún problema): 
- 'Less than 15 minutes a day': 0,
- '15-30 minutes a day': 1,
- '30-60 minutes a day': 2,
- '60-120 minutes a day': 3,
- 'Over 120 minutes a day': 4

LabelEncoder con los valores asignados

In [112]:
df['TimeSearching'].unique()

array([nan, '15-30 minutes a day', 'Over 120 minutes a day',
       '30-60 minutes a day', 'Less than 15 minutes a day',
       '60-120 minutes a day'], dtype=object)

In [113]:
# Primero trato los Nan
df['TimeSearching'] = df['TimeSearching'].fillna('IDK')
print(df['TimeSearching'].isna().sum())

0


In [114]:
labels91 = {
'Less than 15 minutes a day': 0,
    '15-30 minutes a day': 1,
    '30-60 minutes a day': 2,
    '60-120 minutes a day': 3,
    'Over 120 minutes a day': 4,
    'IDK': -1
}
df['TimeSearching'] = df['TimeSearching'].map(labels91).fillna(-1)

92. TimeAnswering (Cuánto tiempo gastas en responder preguntas que te hacen?)

LabelEncoder con los valores asignados a la 91 (mismas respuestas)

In [115]:
df['TimeAnswering'].unique()

array([nan, '30-60 minutes a day', 'Over 120 minutes a day',
       '15-30 minutes a day', '60-120 minutes a day',
       'Less than 15 minutes a day'], dtype=object)

In [116]:
# Primero trato los Nan
df['TimeAnswering'] = df['TimeAnswering'].fillna('IDK')
print(df['TimeAnswering'].isna().sum())

0


In [117]:
df['TimeAnswering'] = df['TimeAnswering'].map(labels91).fillna(-1)

93. Frustration: Qué desafío te causa mas frustración
- Complexity of tech stack for build 
- Complexity of tech stack for deployment 
- Amount of technical debt 
- Number of software tools in use 
- Patching/updating core components 
- Maintaining security of code being produced Maintaining security of systems/platforms used in work 
- Reliability of tools/systems used in work 
- Tracking my work 
- Showing my contributions 
- None of these

Dividirlo por ';' y luego OneHotEncoder. 

94. ProfessionalTech: Irrelevante, lo elimino.

95. ProfessionalCloud: Irrelevante, lo elimino

In [118]:
df=  df.drop(['ProfessionalCloud'], axis=1)

96. ProfessionalQuestion: Irrelevante, lo elimino

In [119]:
df=  df.drop(['ProfessionalQuestion'], axis=1)

97. Industry (Industria de tu compañía):
- Software Development (0)
- Computer Systems Design and Services (0)
- Internet, Telecomm or Information Services (0)
- Fintech (0)
- Energy (3)
- Government (4)
- Banking/Financial Services (1)
- Manufacturing (3)
- Transportation, or Supply Chain (3)
- Healthcare (2)
- Retail and Consumer Services (4)
- Higher Education (2)
- Media & Advertising Services (4)
- Insurance (1)
- Other(4)

Los agrupo y les hago OneHotEncoder. 

In [120]:
df['Industry'].unique()

array([nan, 'Insurance', 'Healthcare',
       'Computer Systems Design and Services',
       'Media & Advertising Services', 'Software Development', 'Fintech',
       'Retail and Consumer Services',
       'Internet, Telecomm or Information Services', 'Other:', 'Energy',
       'Transportation, or Supply Chain', 'Banking/Financial Services',
       'Manufacturing', 'Higher Education', 'Government'], dtype=object)

In [121]:
# Diccionario de mapeo
industry_map = {
    'Software Development': 'Tecnología y Servicios Digitales',
    'Computer Systems Design and Services': 'Tecnología y Servicios Digitales',
    'Internet, Telecomm or Information Services': 'Tecnología y Servicios Digitales',
    'Fintech': 'Tecnología y Servicios Digitales',
    'Energy': 'Industria y Energía',
    'Government': 'Otros Servicios',
    'Banking/Financial Services': 'Servicios Financieros',
    'Manufacturing': 'Industria y Energía',
    'Transportation, or Supply Chain': 'Industria y Energía',
    'Healthcare': 'Salud y Educación',
    'Retail and Consumer Services': 'Otros Servicios',
    'Higher Education': 'Salud y Educación',
    'Media & Advertising Services': 'Otros Servicios',
    'Insurance': 'Servicios Financieros',
    'Other:': 'Otros Servicios'
}

# Función para manejar NaN y aplicar el mapeo
def map_industry(value):
    if pd.isna(value):
        return 'Desconocido'
    return industry_map.get(value, 'Otros Servicios')

# Suponiendo que tienes un DataFrame llamado 'df' con una columna 'Industry'
# Aplicamos el mapeo a la columna 'Industry'
df['Industry_Category'] = df['Industry'].map(industry_map)

# Creamos el OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Aplicamos el OneHotEncoder
encoded = encoder.fit_transform(df[['Industry_Category']])

# Creamos un DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['Industry_Category']), index=df.index)

# Unimos el DataFrame original con las nuevas columnas codificadas
df = pd.concat([df, encoded_df], axis=1)

# Eliminamos la columna temporal 'Industry_Category' si ya no la necesitas
df = df.drop(['Industry_Category', 'Industry', 'Industry_Category_nan'], axis=1)

Por la forma en que OneHotEncoder trata los Nan, se me siguen creando columnas con Industry_Category_nan. Simplemente la Elimino y esa fila quedara sin valores

Creo la funcion para aplicar OneHotEncoder a todas las columnas que quedan, a las que primero tengo que tratar haciendo split de las ;. 

In [122]:
def process_and_encode(df, columns):
    for column in columns:
        # Separamos las categorías por el delimitador ";"
        df[column] = df[column].fillna('').str.split(';')
        
        # Creamos un objeto MultiLabelBinarizer
        mlb = MultiLabelBinarizer()
        
        # Aplicamos MultiLabelBinarizer a las categorías separadas
        encoded_values = mlb.fit_transform(df[column])
        
        # Creamos un DataFrame con los valores codificados
        encoded_df = pd.DataFrame(encoded_values, columns=[f"{column}_{c}" for c in mlb.classes_], index=df.index)
        
        # Añadimos los resultados al DataFrame original
        df = pd.concat([df, encoded_df], axis=1)
        
        # Eliminamos la columna original
        df.drop(column, axis=1, inplace=True)
    
    return df

Los Nan que existan en estas columnas he decidido dejarlos, y cuando se aplique el OneHotEncoder se pondran todos 0 en las columnas que agrega. En caso de que no sea asi, eliminare las columnas que sean nan

In [123]:
# Supongamos que estas son las columnas que contienen valores separados por ';'
columns_to_encode = ['TechDoc', 'BuyNewTool', 'TechEndorse',
    'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 'EmbeddedHaveWorkedWith', 
    'MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackSyncHaveWorkedWith',
    'AISearchDevHaveWorkedWith', 'AIBen', 'AIToolCurrently Using', 'AINextMuch more integrated', 'AIChallenges', 'Frustration'
    ]  # Reemplaza con los nombres reales de las columnas


# Aplicamos la función al DataFrame
df_encoded = process_and_encode(df, columns_to_encode)

In [124]:
columns_to_drop = [
    'LanguageWantToWorkWith', 'LanguageAdmired',
    'DatabaseWantToWorkWith', 'DatabaseAdmired',
    'PlatformWantToWorkWith', 'PlatformAdmired',
    'WebframeWantToWorkWith', 'WebframeAdmired',
    'EmbeddedWantToWorkWith', 'EmbeddedAdmired', 
    'MiscTechWantToWorkWith', 'MiscTechAdmired',
    'ToolsTechWantToWorkWith', 'ToolsTechAdmired', 
    'NEWCollabToolsWantToWorkWith', 'NEWCollabToolsAdmired', 
    'OpSysPersonal use',
    'OfficeStackAsyncWantToWorkWith', 'OfficeStackAsyncAdmired', 
    'OfficeStackSyncWantToWorkWith', 'OfficeStackSyncAdmired',
    'AISearchDevWantToWorkWith', 'AISearchDevAdmired',
    'NEWSOSites', 'SOVisitFreq', 'SOAccount', 'SOPartFreq', 'SOHow', 'SOComm', 
    'AIToolInterested in Using', 'AIToolNot interested in Using',
    'AINextNo change', 'AINextMore integrated', 'AINextLess integrated', 'AINextMuch less integrated',
    'AIEthics', 'TBranch', 'ICorPM', 'WorkExp', 
    'Knowledge_1', 'Knowledge_2', 'Knowledge_3', 'Knowledge_4', 'Knowledge_5', 'Knowledge_6', 'Knowledge_7', 'Knowledge_8', 'Knowledge_9', 
    'ProfessionalTech', 'SurveyLength', 'SurveyEase'
]

98. JobSatPoints_1: Elimino, irrelevante
99. JobSatPoints_4: Elimino, irrelevante

100. JobSatPoints_5: Elimino, irrelevante

101. JobSatPoints_6: Elimino, irrelevante

102. JobSatPoints_7: Elimino, irrelevante
102. JobSatPoints_8: Elimino, irrelevante
103. JobSatPoints_9: Elimino, irrelevante

104. JobSatPoints_10: Elimino, irrelevante
105. JobSatPoints_11: Elimino, irrelevante
106. JobSat: Elimino, irrelevante

In [125]:
df = df.drop(['JobSatPoints_1', 'JobSatPoints_4', 'JobSatPoints_5', 'JobSatPoints_6', 'JobSatPoints_7', 
               'JobSatPoints_8', 'JobSatPoints_9', 'JobSatPoints_10', 'JobSatPoints_11' ,'JobSat'], 
             axis=1)

Elimino las columas que estaban en columns_to_drop

In [126]:
df = df.drop(columns_to_drop, axis=1)

In [127]:
df=df.drop(['TechDoc', 'BuyNewTool', 'TechEndorse',
    'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 'EmbeddedHaveWorkedWith', 
    'MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackSyncHaveWorkedWith',
    'AISearchDevHaveWorkedWith', 'AIBen', 'AIToolCurrently Using', 'AINextMuch more integrated', 'AIChallenges', 'Frustration', 'LearnCodeOnline_Category_nan'
    ], axis=1)

In [128]:
for col in df.columns:
    print(col)

MainBranch
CodingActivities
EdLevel
LearnCode
YearsCode
YearsCodePro
OrgSize
PurchaseInfluence
CompTotal
AISelect
AISent
AIAcc
AIComplex
AIThreat
Frequency_1
Frequency_2
Frequency_3
TimeSearching
TimeAnswering
Age_18-24 years old
Age_25-34 years old
Age_35-44 years old
Age_45-54 years old
Age_55-64 years old
Age_65 years or older
Age_Prefer not to say
Age_Under 18 years old
employment_map
RemoteWork_Hybrid (some remote, some in-person)
RemoteWork_In-person
RemoteWork_Remote
LearnCodeOnline_Category_Technical_Reference
LearnCodeOnline_Category_Visual_Interactive
DevType_Category_Desarrollo de Software
DevType_Category_Gestión y Liderazgo
DevType_Category_Hardware y Blockchain
DevType_Category_Infraestructura y Sistemas
DevType_Category_Ingeniería de Datos y Ciencia de Datos
DevType_Category_Investigación y Educación
DevType_Category_Roles de Soporte y Otros
Android-based
Linux-based
MacOS
Otros
Windows-based
Industry_Category_Industria y Energía
Industry_Category_Otros Servicios
Industr

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Data columns (total 50 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   MainBranch                                               784 non-null    int64  
 1   CodingActivities                                         784 non-null    float64
 2   EdLevel                                                  784 non-null    int64  
 3   LearnCode                                                784 non-null    int64  
 4   YearsCode                                                784 non-null    int32  
 5   YearsCodePro                                             784 non-null    int32  
 6   OrgSize                                                  784 non-null    float64
 7   PurchaseInfluence                                        784 non-null    int64  
 8   CompTotal                     

In [130]:
df.head()

,MainBranch,CodingActivities,EdLevel,LearnCode,YearsCode,YearsCodePro,OrgSize,PurchaseInfluence,CompTotal,AISelect,AISent,AIAcc,AIComplex,AIThreat,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_18-24 years old,Age_25-34 years old,Age_35-44 years old,Age_45-54 years old,Age_55-64 years old,Age_65 years or older,Age_Prefer not to say,Age_Under 18 years old,employment_map,"RemoteWork_Hybrid (some remote, some in-person)",RemoteWork_In-person,RemoteWork_Remote,LearnCodeOnline_Category_Technical_Reference,LearnCodeOnline_Category_Visual_Interactive,DevType_Category_Desarrollo de Software,DevType_Category_Gestión y Liderazgo,DevType_Category_Hardware y Blockchain,DevType_Category_Infraestructura y Sistemas,DevType_Category_Ingeniería de Datos y Ciencia de Datos,DevType_Category_Investigación y Educación,DevType_Category_Roles de Soporte y Otros,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales
0,5,0.0,5,8,14,10,5.0,1,80000.0,1,4,-1,-1,0,-1,-1,-1,-1,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0
1,5,0.0,5,13,15,11,2.0,2,85000.0,1,0,-1,-1,0,1,2,1,1,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0.0,0.0,0.0,1.0,0.0
2,5,-1.0,4,14,30,24,3.0,1,30000.0,2,3,2,2,2,2,1,1,4,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0,0,0,0.0,0.0,1.0,0.0,0.0
3,5,2.0,5,9,30,22,4.0,1,40000.0,2,5,3,3,0,0,0,1,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0.0,0.0,0.0,0.0,1.0
4,4,1.0,6,0,37,31,-1.0,0,50000.0,2,4,1,1,0,-1,-1,-1,-1,-1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,0,0,1,0.0,0.0,0.0,0.0,0.0


## Me traigo el df del 2023, ya limpio y con numeros, para hacerle concat a este

In [131]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df2023.pickle', 'rb') as archivo:
    df1 = pickle.load(archivo)

In [132]:
df1

,MainBranch,CodingActivities,EdLevel,LearnCode,YearsCode,YearsCodePro,OrgSize,PurchaseInfluence,CompTotal,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_18-24 years old,Age_25-34 years old,Age_35-44 years old,Age_45-54 years old,Age_55-64 years old,Age_65 years or older,Age_Under 18 years old,employment_map,"RemoteWork_Hybrid (some remote, some in-person)",RemoteWork_In-person,RemoteWork_Remote,LearnCodeOnline_Category_Technical_Reference,LearnCodeOnline_Category_Visual_Interactive,DevType_Category_Desarrollo de Software,DevType_Category_Gestión y Liderazgo,DevType_Category_Hardware y Blockchain,DevType_Category_Infraestructura y Sistemas,DevType_Category_Ingeniería de Datos y Ciencia de Datos,DevType_Category_Investigación y Educación,DevType_Category_Roles de Soporte y Otros,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales
0,5,2.0,5,7,0,10,1,0,35000.0,2,4,1,1,1,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0.0,0.0,0.0,0.0,1.0
1,5,-1.0,3,6,15,15,8,0,70000.0,1,3,1,1,1,1,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0.0,0.0,0.0,0.0,1.0
2,5,-1.0,4,8,5,3,8,0,25000.0,1,0,0,1,0,2,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0.0,0.0,0.0,0.0,1.0
3,5,0.0,4,4,4,2,5,0,35000.0,2,3,-1,-1,-1,-1,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0
4,5,-1.0,6,5,15,10,6,0,42000.0,2,4,-1,-1,-1,-1,-1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,5,-1.0,4,3,8,6,4,1,51000.0,2,4,1,1,1,1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0
1146,5,-1.0,4,11,8,3,1,2,100000.0,2,5,-1,-1,-1,-1,-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0
1147,5,-1.0,4,4,6,2,1,2,27000.0,2,5,1,1,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0.0,0.0,0.0,0.0,1.0
1148,5,2.0,4,2,12,6,4,0,30000.0,0,0,0,0,0,1,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0.0,0.0,0.0,0.0,1.0


In [133]:
# Combinar los DataFrames usando pd.concat()
df_merged = pd.concat([df1, df], ignore_index=True)

In [134]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1934 entries, 0 to 1933
Data columns (total 50 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   MainBranch                                               1934 non-null   int64  
 1   CodingActivities                                         1934 non-null   float64
 2   EdLevel                                                  1934 non-null   int64  
 3   LearnCode                                                1934 non-null   int64  
 4   YearsCode                                                1934 non-null   int32  
 5   YearsCodePro                                             1934 non-null   int32  
 6   OrgSize                                                  1934 non-null   float64
 7   PurchaseInfluence                                        1934 non-null   int64  
 8   CompTotal                   

In [135]:
df_merged 

,MainBranch,CodingActivities,EdLevel,LearnCode,YearsCode,YearsCodePro,OrgSize,PurchaseInfluence,CompTotal,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_18-24 years old,Age_25-34 years old,Age_35-44 years old,Age_45-54 years old,Age_55-64 years old,Age_65 years or older,Age_Under 18 years old,employment_map,"RemoteWork_Hybrid (some remote, some in-person)",RemoteWork_In-person,RemoteWork_Remote,LearnCodeOnline_Category_Technical_Reference,LearnCodeOnline_Category_Visual_Interactive,DevType_Category_Desarrollo de Software,DevType_Category_Gestión y Liderazgo,DevType_Category_Hardware y Blockchain,DevType_Category_Infraestructura y Sistemas,DevType_Category_Ingeniería de Datos y Ciencia de Datos,DevType_Category_Investigación y Educación,DevType_Category_Roles de Soporte y Otros,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales,AIAcc,AIComplex,AIThreat,Age_Prefer not to say
0,5,2.0,5,7,0,10,1.0,0,35000.0,2,4,1,1,1,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN
1,5,-1.0,3,6,15,15,8.0,0,70000.0,1,3,1,1,1,1,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN
2,5,-1.0,4,8,5,3,8.0,0,25000.0,1,0,0,1,0,2,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN
3,5,0.0,4,4,4,2,5.0,0,35000.0,2,3,-1,-1,-1,-1,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,5,-1.0,6,5,15,10,6.0,0,42000.0,2,4,-1,-1,-1,-1,-1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,5,6.0,6,4,2,1,2.0,0,18000.0,-1,0,-1,-1,-1,-1,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,1.0,0.0
1930,5,-1.0,2,6,10,6,3.0,0,70000.0,0,0,-1,-1,-1,-1,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,1.0,0.0
1931,5,-1.0,-1,7,4,3,4.0,1,22000.0,2,4,-1,-1,-1,-1,-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1932,5,-1.0,4,4,9,7,-1.0,2,81600.0,2,5,1,1,1,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0.0,0.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0


Ya tengo toda la data lista para terminar de hacer feature enegineering y empezar con el modelo. 
Lo exporto, sigo en otro notebook

In [136]:
# Guardar el DataFrame en un archivo
with open('df_final.pickle', 'wb') as archivo:
    pickle.dump(df_merged, archivo)